<a href="https://colab.research.google.com/github/erviloria4/Solution-Starter-Kit-Communication-2020/blob/master/Copia_en_Espa%C3%B1ol_de_tutorial_great_expectations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original: https://colab.research.google.com/github/datarootsio/tutorial-great-expectations/blob/main/tutorial_great_expectations.ipynb

# Tutorial de Great Expectations

¡Bienvenidos! En este tutorial echaremos un vistazo a Great Expectations, una herramienta escrita y configurada en Python que te ayuda a mantener un control sobre la calidad de tus datos. Ofrece una solución completa para probar y documentar tus datos, de modo que nadie tenga sorpresas al utilizarlos. Para lograr esto, creas suites de expectativas. Puedes pensar en ellas como pruebas unitarias, pero para datos. También sirven como documentación para tu conjunto de datos, para que no tengas que repetirte.

¿A qué nos referimos con calidad de datos? Bueno, los datos de mala calidad pueden ocurrir por diferentes razones. Usualmente, los datos tienen mala calidad si su estructura (por ejemplo, las columnas y sus tipos en una tabla) o su contenido (celdas específicas en una tabla) no son lo que esperabas.

Para más información sobre Great Expectations y los problemas que resuelve, podemos recomendar el blogpost de los autores: [Down with Pipeline debt / Introducing Great Expectations](https://medium.com/@expectgreatdata/down-with-pipeline-debt-introducing-great-expectations-862ddc46782a). ¡Es una buena lectura!

## ¿Qué es exactamente Great Expectations?

<img src='https://github.com/datarootsio/tutorial-great-expectations/blob/main/figures/in_out.png?raw=1' width=800px>

Great Expectations se puede utilizar con sus activos de datos existentes - es capaz de utilizar diferentes backends como bases de datos SQL, clusters Spark, o simplemente su viejo sistema de archivos. Ejecutará sus conjuntos de expectativas en estos backends y generará informes sobre los resultados de su validación.

Escribir tu suite de expectativas se hace normalmente a través de cuadernos Jupyter, así que te sentirás como en casa. Este cuaderno sería un ejemplo de cómo funciona.


## En este tutorial
Te daremos una breve introducción a los conceptos principales utilizados en Great Expectations, guiándote en la escritura de tus primeras expectativas y generando tu primer informe de datos. Hemos añadido muchas referencias a la documentación oficial que puedes consultar cuando estés configurando tu propio entorno.

Contenidos:
* [Comenzando](#section-getting-started)
* [La Suite de Expectativas](#section-expectation-suite)
* [Documentos de Datos](#section-data-docs)
* [Contexto de Datos](#section-data-context)
* [Puntos de Control](#section-checkpoints)
* [Perfilado de Datos](#section-profiling)
* [La CLI de Great Expectations](#section-cli)
* [Configurando tu propio proyecto](#section-setup)

## Ejecución en Google Colab
Si estás ejecutando esto en Google Colab, asegúrate de ejecutar la celda a continuación para configurar todo.

In [ ]:
%%bash
if [[ ! -d great_expectations ]]
then
  git init
  git remote add origin https://github.com/datarootsio/tutorial-great-expectations.git
  git pull origin main
  apt-get install tree
  mkdir -p great_expectations/checkpoints
fi

In [ ]:
!pip install great_expectations=="0.15.18"

## Getting started

¡Entonces, comencemos!

In [ ]:
import great_expectations as ge

Primero necesitaremos un `DataContext`. Esto representa un proyecto de Great Expectations, conteniendo todas tus configuraciones, suites de expectativas, fuentes de datos, etc. Examinaremos más detenidamente el contexto de datos más adelante [Contexto de Datos](#section-data-context), pero solo para comenzar, hemos incluido uno simple con este tutorial.

Vamos a cargar ese ahora mismo. Por defecto, Great Expectations buscará tu configuración en el directorio `great_expectations`.

In [ ]:
context = ge.data_context.DataContext()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Ahora que tenemos nuestro `DataContext` listo, podemos añadir una suite de expectativas. Piensa en esto como una suite de pruebas, pero para tus datos en lugar de para tu código. Normalmente harías esto a través de la CLI, pero hablaremos de eso más adelante [La CLI de Great Expectations]. Nombraremos la suite `check_avocado_data`.








In [ ]:
suite = context.create_expectation_suite(
    'check_avocado_data',
    overwrite_existing=True
)

A continuación, cargamos nuestro conjunto de datos, `avocado.csv`, desde nuestro contexto de datos. Esto implica un poco de configuración, pero no te preocupes demasiado por ahora. Volveremos a eso más tarde [Contexto de Datos](#section-data-context).








In [ ]:
batch_kwargs = {
    'path': 'data/avocado.csv',
    'datasource': 'data_dir',
    'data_asset_name': 'avocado',
    'reader_method': 'read_csv',
    'reader_options': {
        'index_col': 0,
    }
}
batch = context.get_batch(batch_kwargs, suite)

¡Perfecto, eso es todo para la configuración!

Continuemos con nuestros datos de ventas de aguacate.

In [ ]:
batch.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


Esta es la documentación que acompaña a los datos:
- Date - La fecha de la observación
- AveragePrice - el precio promedio de un solo aguacate
- type - tipo de agricultura: convencional u orgánica
- Region - la ciudad o región de la observación
- Total Volume - Número total de aguacates vendidos
- 4046 - Número total de aguacates con PLU 4046 vendidos (Hass pequeño)
- 4225 - Número total de aguacates con PLU 4225 vendidos (Hass grande)
- 4770 - Número total de aguacates con PLU 4770 vendidos (Hass extra grande)

Estas descripciones definitivamente nos ayudan a entender un poco mejor el conjunto de datos, pero no proporcionan muchas garantías. Al consumir este conjunto de datos, ¿qué expectativas podemos tener? ¿El campo `region` siempre estará especificado? ¿El campo `Date` siempre estará en el mismo formato? Esos conteos de ventas, ¿se supone que deben sumar?

Great Expectations nos ayuda a codificar estas propiedades en un conjunto de `Expectations`. Una Expectativa (`Expectation`) es, bueno, algo que esperas que sea cierto en tus datos. Nuevamente, piénsalo como una prueba unitaria para tu conjunto de datos.

Comencemos con una `Expectation` básica. Queremos verificar si nuestra expectativa de que la columna `Date` está presente se cumple.

In [ ]:
batch.expect_column_to_exist('Date')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{
  "expectation_config": {
    "expectation_type": "expect_column_to_exist",
    "kwargs": {
      "column": "Date",
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {},
  "meta": {},
  "success": true
}

El `dict` resultante que recibimos podría parecer un poco extraño al principio, pero más adelante verás cómo se usa esta salida para generar informes [[Documentos de Datos]](#section-data-docs). Por ahora, simplemente observa que `success` tiene el valor `true`, ¡indicando que nuestra expectativa se cumplió!








Este fue un chequeo simple que solo evalúa la forma de los datos, pero no toca los valores en sí (es un _chequeo a nivel de tabla_ o _table-level check_).

Ahora intentemos agregar un chequeo para los valores. Tal vez podamos abordar una de las preocupaciones que planteamos: ¿podemos añadir una `Expectation` que asegure que cada registro tenga su `region` especificada?

In [ ]:
batch.expect_column_values_to_not_be_null('region')

{
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "region",
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 18249,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "success": true
}

¡Funcionó! Esta vez recibimos un poco más de información: la sección `result` ahora contiene algunas métricas sobre nuestros datos. Podemos ver que los 18,249 registros pasaron la verificación y no hubo valores inesperados (es decir, `null`). Si Great Expectations encuentra algún valor problemático, estos serán listados en `partial_unexpected_list`.

Now let's do something that's a bit more strict. It would be nice, for example, to make sure that all `region`s are actually strings, so that we don't end up with numeric regions. Note that the type you specify here should match your backend - you can't expect a spark backend to have PostgresQL types. Refer to to see what type name you should use.

Ahora hagamos algo un poco más estricto. Sería bueno, por ejemplo, asegurarse de que todas las `region`s sean realmente cadenas de texto, para no terminar con regiones numéricas. Ten en cuenta que el tipo que especifiques aquí debe coincidir con tu backend: no puedes esperar que un backend de Spark tenga tipos de PostgresQL. Consulta [la documentación](https://docs.greatexpectations.io/en/latest/autoapi/great_expectations/expectations/core/expect_column_values_to_be_of_type/index.html) para ver qué nombre de tipo deberías usar.

In [ ]:
batch.expect_column_values_to_be_of_type('region', 'str')

{
  "expectation_config": {
    "expectation_type": "_expect_column_values_to_be_of_type__map",
    "kwargs": {
      "column": "region",
      "type_": "str",
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 18249,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "success": true
}

Note that metrics on the amount of missing values were still collected. This way, we can disambiguate between missing values and incorrect values. In case you were wondering, the `unexpected_percent_nonmissing` refers to the percentage of present (non-null) values that did not meet our expectation (they were not a string). If other metrics are unclear to you, check out [this documentation page](https://docs.greatexpectations.io/en/latest/reference/core_concepts/expectations/result_format.html#behavior-for-summary).

Ten en cuenta que todavía se recopilaron métricas sobre la cantidad de valores faltantes. De esta manera, podemos distinguir entre valores faltantes e incorrectos. En caso de que te lo estés preguntando, el `unexpected_percent_nonmissing` se refiere al porcentaje de valores presentes (no nulos) que no cumplieron con nuestra expectativa (no eran una cadena de texto). Si otras métricas no te quedan claras, consulta [esta página de documentación](https://docs.greatexpectations.io/en/latest/reference/core_concepts/expectations/result_format.html#behavior-for-summary).

Ahora que hemos cubierto los conceptos básicos, pasemos a algunas expectativas más sofisticadas. Por ejemplo, podríamos asegurarnos de que todas las `Date`s estén en el formato esperado

In [ ]:
batch.expect_column_values_to_match_strftime_format('Date', "%Y-%m-%d")

{
  "expectation_config": {
    "expectation_type": "expect_column_values_to_match_strftime_format",
    "kwargs": {
      "column": "Date",
      "strftime_format": "%Y-%m-%d",
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 18249,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "success": true
}

Otro ejemplo: podemos asegurarnos de que todos los precios de aguacates listados sean razonables.

In [ ]:
batch.expect_column_values_to_be_between('AveragePrice', min_value=0.5, max_value=3.0)

{
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "AveragePrice",
      "min_value": 0.5,
      "max_value": 3.0,
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 18249,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 11,
    "unexpected_percent": 0.06027727546714888,
    "unexpected_percent_total": 0.06027727546714888,
    "unexpected_percent_nonmissing": 0.06027727546714888,
    "partial_unexpected_list": [
      0.49,
      0.46,
      3.03,
      3.12,
      3.25,
      0.44,
      0.49,
      0.48,
      3.05,
      3.04,
      3.17
    ]
  },
  "meta": {},
  "success": false
}

¡Vaya! Eso falló. ¡Parece que tenemos algunos valores atípicos aquí! Great Expectations los recopiló útilmente para nosotros. Por defecto, recogerá hasta 20 ejemplos de valores que no cumplieron con la expectativa (por eso se llama la lista _parcial_ de inesperados).

Si queremos permitir estos valores atípicos, podemos agregar algo de tolerancia a la verificación utilizando el parámetro `mostly`. Reemplacemos esa expectativa con una nueva, que solo espera que el 99% de los aguacates tengan un precio dentro del rango que especificamos.

In [ ]:
batch.expect_column_values_to_be_between('AveragePrice', min_value=0.5, max_value=3.0, mostly=0.99)

{
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "AveragePrice",
      "min_value": 0.5,
      "max_value": 3.0,
      "mostly": 0.99,
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 18249,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 11,
    "unexpected_percent": 0.06027727546714888,
    "unexpected_percent_total": 0.06027727546714888,
    "unexpected_percent_nonmissing": 0.06027727546714888,
    "partial_unexpected_list": [
      0.49,
      0.46,
      3.03,
      3.12,
      3.25,
      0.44,
      0.49,
      0.48,
      3.05,
      3.04,
      3.17
    ]
  },
  "meta": {},
  "success": true
}

Otro caso de uso común sería cuando solo esperas que un cierto conjunto de valores aparezca en una columna. Este es el caso de nuestra columna `type`, ya que solo conocemos aguacates cultivados `conventional` (convencionales) y `organic` (orgánicos). Añadamos una verificación para eso:

In [ ]:
batch.expect_column_distinct_values_to_be_in_set('type', ['conventional', 'organic'])

{
  "expectation_config": {
    "expectation_type": "expect_column_distinct_values_to_be_in_set",
    "kwargs": {
      "column": "type",
      "value_set": [
        "conventional",
        "organic"
      ],
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": [
      "conventional",
      "organic"
    ],
    "element_count": 18249,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "success": true
}

Incluso podríamos añadir una verificación sobre las frecuencias de los valores. Por ejemplo, si queremos que la proporción de aguacates orgánicos respecto a los convencionales sea aproximadamente igual, podríamos verificar la [divergencia de Kullback-Leiber](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) entre nuestra distribución asumida y la que se observa en el conjunto de datos.

In [ ]:
partition_object = {
    'values': ['conventional', 'organic'],
    'weights': [0.5, 0.5],

}
batch.expect_column_kl_divergence_to_be_less_than('type', partition_object, 0.1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{
  "expectation_config": {
    "expectation_type": "expect_column_kl_divergence_to_be_less_than",
    "kwargs": {
      "column": "type",
      "partition_object": {
        "values": [
          "conventional",
          "organic"
        ],
        "weights": [
          0.5,
          0.5
        ]
      },
      "threshold": 0.1,
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": 1.351245850704074e-08,
    "element_count": 18249,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "success": true
}

Ahora que hemos explorado algunas expectativas, ¿qué tal si intentas agregar una por tu cuenta? Puedes consultar el [glosario de expectativas](https://docs.greatexpectations.io/en/latest/reference/glossary_of_expectations.html) para ver una lista completa de lo que puedes hacer. ¡Desata tu creatividad!

In [ ]:
# El escenario es todo tuyo

<a id="section-expectation-suite"></a>
## The Expectation Suite

Así que, mientras experimentábamos anteriormente, Great Expectations recordó todas las expectativas que ejecutamos. Ahora podemos recuperar el contenido de la suite de la siguiente manera:

In [ ]:
batch.get_expectation_suite()

{
  "expectations": [
    {
      "expectation_type": "expect_column_to_exist",
      "kwargs": {
        "column": "Date"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_not_be_null",
      "kwargs": {
        "column": "region"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_be_of_type",
      "kwargs": {
        "column": "region",
        "type_": "str"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "Date",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_be_between",
      "kwargs": {
        "column": "AveragePrice",
        "min_value": 0.5,
        "max_value": 3.0,
        "mostly": 0.99
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_distinct_values_to_be_in_set",
      "kwargs": {
 

Eso nos dio la representación en `dict` que Great Expectations utiliza internamente para llevar un registro de nuestra suite de expectativas. ¿Puedes reconocer algunas de las expectativas que escribimos?

Una suite de expectativas es simplemente una secuencia de expectativas, como se muestra a continuación.

<img src="https://github.com/datarootsio/tutorial-great-expectations/blob/main/figures/expectation_suite.png?raw=1">

Esta representación luego puede ser guardada en un archivo, para que podamos cargarla nuevamente en otro momento, sin depender del código de Python que la produjo.

Nota que por defecto, las expectativas que fallaron en el `batch` contra el que se ejecutaron serán omitidas. Si quieres incluirlas de todos modos, podrías agregar el parámetro `discard_failed_expectations=False`.

In [ ]:
batch.save_expectation_suite()

¿Qué hizo ese comando? Abramos nuestra carpeta de configuración para intentar encontrar nuestra suite de expectativas.

In [ ]:
!tree great_expectations -nI "uncommitted"

great_expectations
├── checkpoints
├── expectations
│   └── check_avocado_data.json
├── great_expectations.yml
├── notebooks
│   ├── pandas
│   │   └── validation_playground.ipynb
│   ├── spark
│   │   └── validation_playground.ipynb
│   └── sql
│       └── validation_playground.ipynb
└── plugins
    └── custom_data_docs
        └── styles
            └── data_docs_custom_styles.css

9 directories, 6 files


Volveremos a la configuración en un minuto [[Data Context]](#section-data-context), así que no te confundas con esto todavía.

Como puedes ver, el comando `save_expectation_suite` guardó nuestra suite `check_avocado_data` en la carpeta `expectations`. Eso es todo lo que hay, la suite de expectativas es simplemente un archivo json. Contiene la misma representación interna que recuperamos de `get_expectation_suite()`. Puedes revisarlo si lo deseas.

In [ ]:
!cat great_expectations/expectations/check_avocado_data.json

{
  "data_asset_type": "Dataset",
  "expectation_suite_name": "check_avocado_data",
  "expectations": [
    {
      "expectation_type": "expect_column_to_exist",
      "kwargs": {
        "column": "Date"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_not_be_null",
      "kwargs": {
        "column": "region"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_be_of_type",
      "kwargs": {
        "column": "region",
        "type_": "str"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "Date",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_be_between",
      "kwargs": {
        "column": "AveragePrice",
        "max_value": 3.0,
        "min_value": 0.5,
        "mostly": 0.99
      },
      "meta": {}
    },
    {
      "ex


Las expectativas se almacenan en el _almacén de expectativas_ (_expectation store_), que por defecto es la carpeta `expectations` dentro de tu configuración, pero también puedes usar otros sistemas de almacenamiento, como una base de datos SQL o almacenamiento en la nube (S3, Azure Blob Storage o GCS). Consulta almacenes de metadatos para más información.

<a id="validation-results"></a>
Ahora que hemos añadido nuestra suite de expectativas a nuestro `DataContext`, podemos intentar ejecutar la suite completa.

Validar tus datos contra una suite de expectativas se realiza ejecutando un **operador de validación**. Un operador de validación describe qué se debe hacer con los resultados de la validación. Aquí, nos gustaría almacenar los resultados en el disco y generar un informe amigable sobre ellos. Te mostraremos cómo se configura esto en la sección de [Data Context section](#section-data-context), mientras tanto usaremos `my_validation_operator`, que hemos incluido con la configuración.

In [ ]:
results = context.run_validation_operator('my_validation_operator', assets_to_validate=[batch])

ERROR:great_expectations.validation_operators.validation_operators:Error running action with name store_validation_result
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/great_expectations/validation_operators/validation_operators.py", line 458, in _run_actions
    action_result = self.actions[action["name"]].run(
  File "/usr/local/lib/python3.10/dist-packages/great_expectations/checkpoint/actions.py", line 72, in run
    return self._run(
  File "/usr/local/lib/python3.10/dist-packages/great_expectations/checkpoint/actions.py", line 826, in _run
    return_val = self.target_store.set(
  File "/usr/local/lib/python3.10/dist-packages/great_expectations/data_context/store/store.py", line 169, in set
    self.key_to_tuple(key), self.serialize(value), **kwargs
  File "/usr/local/lib/python3.10/dist-packages/great_expectations/data_context/store/validations_store.py", line 172, in serialize
    return self._expectationSuiteValidationResultSchema.dumps(
  

AttributeError: 'Series' object has no attribute 'iteritems'

Una ejecución de validación puede incluir múltiples lotes y suites de expectativas. De esta manera, es posible probar varios archivos en la misma ejecución. Compara esto con cómo una ejecución de tu suite de pruebas puede probar varios módulos de software.

No especificamos explícitamente la suite de expectativas a usar con nuestro lote de datos, porque el `batch` lleva un registro de la suite de expectativas para nosotros. Ya vimos esto cuando recuperamos la suite de ella al principio de esta sección.

Ahora que hemos pasado por eso, echemos un vistazo a los resultados.

In [ ]:
results

NameError: name 'results' is not defined

Esto se llama un _resultado de validación_. Los resultados de validación se guardan en el almacén de validaciones, que por defecto es el directorio `great_expectations/uncommitted/validations`.

In [ ]:
!tree -n great_expectations/uncommitted/validations

great_expectations/uncommitted/validations

0 directories, 0 files


Great Expectations también te permite configurar otros sistemas de almacenamiento como almacén de validaciones, como tu servicio de almacenamiento en la nube favorito o una base de datos SQL. ¡Consulta [almacenes de metadatos](https://docs.greatexpectations.io/en/latest/guides/how_to_guides/configuring_metadata_stores.html) si deseas aprender más!











<a id="section-data-docs"></a>
## Data Docs

Podemos renderizar estos resultados en un informe amigable, llamado un documento de datos (data doc). Estos documentos de datos describirán las expectativas que los datos deben cumplir, así como las métricas que detallan cuán bien los datos cumplen con los requisitos. Así es como Great Expectations combina las pruebas con la documentación.

Recuerda que ya construimos los documentos de datos usando `my_validation_operator` en la sección anterior. ¡Vamos a revisarlos ahora! Te llevaremos a la página de índice, asegúrate de explorar un poco. En la pestaña `Validation Results` encontrarás la ejecución de validación que realizamos anteriormente. Haz clic en ella para obtener un informe amigable sobre sus resultados. En la pestaña `Expectation Suites`, puedes encontrar un documento que detalla las expectativas establecidas por nuestra suite `check_avocado_data`. Verás las expectativas que ejecutamos anteriormente reflejadas en las diferentes secciones.

Si estás ejecutando el tutorial en tu sistema operativo, ejecuta este comando para abrir los documentos de datos:

In [ ]:
context.open_data_docs()

Si estás ejecutando en Docker, prueba este enlace [aquí](/view/great_expectations/uncommitted/data_docs/local_site/index.html). Si los enlaces no funcionan en tu navegador, podrías intentar usar el [navegador de archivos de jupyter](/tree/great_expectations/uncommitted/data_docs/local_site). No es ideal, pero funciona.

De lo contrario, puedes ver los resultados de nuestra ejecución [aquí](https://datarootsio.github.io/tutorial-great-expectations/validation).

Al igual que con los resultados de validación, se pueden configurar diferentes sistemas de almacenamiento para tus documentos de datos. Por ejemplo, podrías alojarlos en un almacenamiento en la nube para facilitar su visualización. Consulta [configurando documentos de datos](https://docs.greatexpectations.io/en/latest/guides/how_to_guides/configuring_data_docs.html) para obtener más información.

<a id="section-data-context"></a>
## Data Context

Antes de continuar, tomemos un momento para observar el `DataContext`, que representa tu configuración de Great Expectations. Consiste en un directorio que contiene archivos de configuración, llamado `great_expectations` por defecto.

Nota: estamos omitiendo el directorio `uncommitted` aquí. Contiene archivos de salida (como documentos de datos renderizados), que no son parte de la configuración.

In [ ]:
!tree great_expectations -nI 'uncommitted'

great_expectations
├── checkpoints
├── expectations
│   └── check_avocado_data.json
├── great_expectations.yml
├── notebooks
│   ├── pandas
│   │   └── validation_playground.ipynb
│   ├── spark
│   │   └── validation_playground.ipynb
│   └── sql
│       └── validation_playground.ipynb
└── plugins
    └── custom_data_docs
        └── styles
            └── data_docs_custom_styles.css

9 directories, 6 files


La configuración principal se encuentra en `great_expectations.yml`. No entraremos en todos los detalles aquí, puedes referirte a la [referencia de contexto de datos para eso](https://docs.greatexpectations.io/en/latest/reference/spare_parts/data_context_reference.html). En cambio, solo introduciremos algunos conceptos con los que querrás familiarizarte:

- Una **fuente de datos** (**data source**) es algo que puede proporcionar datos a Great Expectations, como una base de datos SQL.
- Un **activo de datos** (**data asset**) es un conjunto de datos que reside en una fuente de datos, como una tabla SQL.

En la configuración que proporcionamos, hay una *fuente de datos* llamada `data_dir`, que es simplemente una carpeta con archivos csv dentro. El archivo `avocado.csv` con el que estamos trabajando sería un activo de datos. Puedes encontrar más información sobre las fuentes de datos en la [referencia de contexto de datos](https://docs.greatexpectations.io/en/latest/reference/spare_parts/data_context_reference.html#datasources). Para configurar las tuyas, consulta las [guías de configuración de fuentes de datos](https://docs.greatexpectations.io/en/latest/guides/how_to_guides/configuring_datasources.html).

- Un **operador de validación** (**validation operator**) especifica qué se debe hacer con los resultados de tu validación. Algunos ejemplos podrían ser escribir los resultados de la validación en una base de datos, publicar documentos de datos o enviar una notificación a un canal de Slack.
Si deseas saber más, puedes consultar las páginas de documentación de [operadores de validación y acciones](https://docs.greatexpectations.io/en/latest/reference/core_concepts/validation_operators_and_actions.html) y [cómo agregar un operador de validación](https://docs.greatexpectations.io/en/latest/guides/how_to_guides/validation/how_to_add_a_validation_operator.html).

- Los almacenes (o **stores**) pueden usarse para configurar cómo se almacenarán los datos de expectativas y validación. Consulta [configuración de almacenes de metadatos](https://docs.greatexpectations.io/en/latest/guides/how_to_guides/configuring_metadata_stores.html) si estás interesado.

Todos estos están configurados en el archivo `great_expectations.yml`. Echaremos un breve vistazo a su contenido ahora, pero no te preocupes demasiado, esto está aquí solo con fines ilustrativos.

In [ ]:
!cat great_expectations/great_expectations.yml

# Welcome to Great Expectations! Always know what to expect from your data.
#
# Here you can define datasources, batch kwargs generators, integrations and
# more. This file is intended to be committed to your repo. For help with
# configuration please:
#   - Read our docs: https://docs.greatexpectations.io/en/latest/how_to_guides/spare_parts/data_context_reference.html#configuration
#   - Join our slack channel: http://greatexpectations.io/slack

# config_version refers to the syntactic version of this config file, and is used in maintaining backwards compatibility
# It is auto-generated and usually does not need to be changed.
config_version: 2.0

# Datasources tell Great Expectations where your data lives and how to get it.
# You can use the CLI command `great_expectations datasource new` to help you
# add a new datasource. Read more at https://docs.greatexpectations.io/en/latest/reference/core_concepts/datasource_reference.html
datasources:
  data_dir:
    batch_kwargs_generators:
 

Además, también tenemos dos directorios importantes: `expectations`, que contiene nuestras suites de expectativas, y `checkpoints`, que revisaremos a continuación.

El siguiente diagrama muestra una representación de nuestro contexto de datos.
<img src="https://github.com/datarootsio/tutorial-great-expectations/blob/main/figures/data_context.png?raw=1" width=800px>

<a id="section-checkpoints"></a>
## Checkpoints

Recuerda cómo lanzamos una ejecución de validación en la sección [Expectation Suite](#section-expectation-suite). Allí, escribimos código para ejecutar la validación en el lote de datos y la suite de expectativas que definimos anteriormente. Si agrupamos todos estos parámetros de ejecución en un solo archivo de configuración, podríamos volver a ejecutar fácilmente la validación, por ejemplo, cada vez que cambien nuestros datos. Dicho archivo de configuración se llama `Checkpoint` en Great Expectations.

Como recordatorio rápido, para ejecutar una validación necesitamos:
- Un *operador de validación* (o *validation operator*) para manejar los resultados de la validación.
- Una lista de *batches*, cada uno compuesto por
  - Un lote de datos a verificar
  - Suites de expectativas para comparar

Para crear un checkpoint, simplemente creamos un archivo en el directorio `checkpoints` de nuestra configuración de great_expectations. Crearemos el archivo manualmente ahora para fines de demostración, pero al hacerlo en tu propio proyecto probablemente querrás usar la CLI [[The Great Expectations CLI]](#section-cli), que te guiará en el proceso.

In [ ]:
%%writefile great_expectations/checkpoints/avocado_data.yml

validation_operator_name: my_validation_operator
batches:
  - batch_kwargs:
      path: data/avocado.csv
      datasource: data_dir
      data_asset_name: avocado
      reader_method: read_csv
      reader_options:
        index_col: 0
    expectation_suite_names:
      - check_avocado_data

Writing great_expectations/checkpoints/avocado_data.yml


La propiedad `batch_kwargs` especifica cómo debe cargarse el activo de datos. Es posible que reconozcas los parámetros de cuando cargamos por primera vez el archivo `avocado.csv`.

Este también podría ser un buen momento para señalar que nuestro lote de datos se leerá mediante pandas internamente (lo configuramos en la fuente de datos `data_dir`). En `batch_kwargs`, especificamos que nos gustaría usar el método `read_csv` de pandas, que recibirá el diccionario `reader_options` como parámetros adicionales.

Para obtener más información sobre los lotes, consulta la guía [creando batches](https://docs.greatexpectations.io/en/latest/guides/how_to_guides/creating_batches.html).

El checkpoint se puede ejecutar usando la CLI de great_expectations:

In [ ]:
!great_expectations checkpoint run avocado_data

Using v3 (Batch Request) API
Checkpoint store named 'checkpoint_store' is not a configured store, so will try to use default Checkpoint store.
  Please update your configuration to the new version number 3.0 in order to use the new 'Checkpoint Store' feature.
  Visit https://docs.greatexpectations.io/docs/guides/miscellaneous/migration_guide#migrating-to-the-batch-request-v3-api to learn more about the upgrade process.
Error running action with name store_validation_result
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/great_expectations/validation_operators/validation_operators.py", line 458, in _run_actions
    action_result = self.actions[action["name"]].run(
  File "/usr/local/lib/python3.10/dist-packages/great_expectations/checkpoint/actions.py", line 72, in run
    return self._run(
  File "/usr/local/lib/python3.10/dist-packages/great_expectations/checkpoint/actions.py", line 826, in _run
    return_val = self.target_store.set(
  File "/usr/lo

Para resumir: un checkpoint es una _verificación ejecutable_ para tus datos. Son tu primer paso para integrar Great Expectations en tus canalizaciones y flujos de trabajo.

Para obtener más información sobre cómo hacerlo, consulta las [guías de validación](https://docs.greatexpectations.io/en/latest/guides/how_to_guides/validation.html) o las [guías de flujos de trabajo y patrones](https://docs.greatexpectations.io/en/latest/guides/workflows_patterns.html).

Los checkpoints y los lotes se representan visualmente a continuación.

<img src="https://github.com/datarootsio/tutorial-great-expectations/blob/main/figures/checkpoint.png?raw=1" width=600px>
<img src="https://github.com/datarootsio/tutorial-great-expectations/blob/main/figures/batch.png?raw=1" width=600px>

<a id="section-profiling"></a>
## Profiling

En las secciones anteriores, exploramos cómo obtener algunas métricas sobre nuestros datos utilizando expectativas. Pero, ¿qué pasa si no sabes exactamente qué esperar de tus datos? Bueno, podrías intentar usar la función de perfilado de Great Expectations, que puede intentar extraer algunas métricas útiles de tus datos. Para probar el perfilado de nuestra fuente de datos `data_dir` preconfigurada, podemos usar la CLI:

In [ ]:
!great_expectations datasource profile data_dir -y

Using v3 (Batch Request) API
Usage: great_expectations datasource [OPTIONS] COMMAND [ARGS]...
Try 'great_expectations datasource --help' for help.

Error: No such command 'profile'.


Si estás ejecutando en tu propio sistema operativo, ejecutar ese comando debería haber abierto los documentos de datos recién creados en tu navegador. Si no, puedes [ver los resultados de nuestra ejecución](https://datarootsio.github.io/tutorial-great-expectations/profiling). Puedes encontrar los resultados en la pestaña `Profiling Results`. El perfilador también generó una suite de expectativas basada en sus observaciones, que puedes encontrar en la pestaña `Expectation Suites`. Ten en cuenta que esta es una función experimental y la suite generada generalmente no es tan útil, pero podría ser un buen punto de partida para escribir las tuyas propias.

Si deseas saber más sobre el perfilado, la [referencia de perfilado](https://docs.greatexpectations.io/en/latest/reference/spare_parts/profiling_reference.html) puede ayudarte.

<a id="section-cli"></a>
## The Great Expectations CLI

Para los propósitos de este tutorial, principalmente interactuamos directamente con Great Expectations. Si vas a configurar y usar Great Expectations por ti mismo, te recomendamos usar la CLI tanto como sea posible. Los conceptos deberían serte familiares a estas alturas: consulta la [guía de la CLI](https://docs.greatexpectations.io/en/latest/guides/how_to_guides/miscellaneous/command_line.html) para más información.

In [ ]:
!great_expectations --help

Usage: great_expectations [OPTIONS] COMMAND [ARGS]...

  Welcome to the great_expectations CLI!

  Most commands follow this format: great_expectations <NOUN> <VERB>

  The nouns are: checkpoint, datasource, docs, init, project, store, suite,
  validation-operator. Most nouns accept the following verbs: new, list, edit

Options:
  --version                Show the version and exit.
  --v3-api / --v2-api      Default to v3 (Batch Request) API. Use --v2-api for
                           v2 (Batch Kwargs) API
  -v, --verbose            Set great_expectations to use verbose output.
  -c, --config TEXT        Path to great_expectations configuration file
                           location (great_expectations.yml). Inferred if not
                           provided.
  -y, --assume-yes, --yes  Assume "yes" for all prompts.
  --help                   Show this message and exit.

Commands:
  checkpoint  Checkpoint operations
  datasource  Datasource operations
  docs        Data Docs operati

<a id="section-setup"></a>
## Setting up your own project

Para iniciar tu propio proyecto, ejecuta `great_expectations init` y sigue las instrucciones. Esto te proporcionará una configuración sencilla, como la que proporcionamos.

Una vez que hayas creado tu suite usando `rgeat_expectations suite new`, puedes usar el comando `great_expectations suite edit` para abrir un cuaderno autogenerado que puedes usar para configurar tu suite. ¡Deberías ser capaz de reconocer la estructura de la primera parte de este cuaderno! ;-)

La [guía para empezar](https://docs.greatexpectations.io/en/latest/guides/tutorials/getting_started.html) puede ayudarte en el camino. Para obtener ideas sobre cómo Great Expectations puede integrarse en tu flujo de trabajo, consulta [Patrones de despliegue](https://docs.greatexpectations.io/en/latest/reference/core_concepts/validation.html#deployment-patterns).



<a id="section-conclusion"></a>
## Final words
Para recapitular, en este cuaderno tutorial, comenzamos dándote una visión general de la herramienta y su propósito. Luego te mostramos cómo comenzar con la biblioteca de Python y definir tus expectativas. Vimos que las expectativas pueden agruparse como suites, que pueden usarse con operadores de validación para producir resultados de validación. Echamos un vistazo a los documentos de datos, una forma clara de visualizar tus resultados y documentación de datos. Luego profundizamos en el contexto de datos, mostrando cómo se configura la herramienta. Echamos un vistazo a los checkpoints, que te permiten automatizar tus pruebas de datos. Hablamos un poco sobre el perfilado, una función experimental para generar expectativas a partir de datos dados. Finalmente, te presentamos la CLI y te encaminamos para que comiences a usar Great Expectations de inmediato.

¡Esperamos que hayas disfrutado del tutorial y te deseamos lo mejor en el uso de Great Expectations con tus proyectos!

¿Interesado en soporte? No dudes en contactarnos a info@dataroots.io (¡en inglés!).